# Connexion au cluster

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Airplane ML") \
    .config('spark.executor.memory', '6g') \
    .config('spark.driver.memory', '2g') \
    .getOrCreate()

# Imports

In [ ]:
from pyspark.sql import *
from pyspark.sql.types import *
import pyspark.sql.functions as F

# Chargement des données

In [ ]:
flights = spark.read.parquet('/data/parquet/flights')

In [ ]:
#flights.groupBy(flights.Origin).count().limit(10).toPandas()

In [ ]:
flights = flights.where((flights.Origin == 'ORD')) \
    .select('Year', 'Month', 'DayOfWeek', 'CRSDepTime', 'UniqueCarrier', 'FlightNum', 'Origin', 'Dest', 'ArrDelay') \
    .na.drop() \
    .withColumn('Delayed', (F.when(flights.ArrDelay == 'NA', 0).otherwise(flights.ArrDelay).cast('integer') > 20).cast('string')) \
    .withColumn('DepTime', flights.CRSDepTime.cast('integer'))
    
features = ['Month', 'DayOfWeek', 'DepTime', 'UniqueCarrierIndex', 'FlightNumIndex', 'OriginIndex', 'DestIndex']

In [ ]:
flights.filter(flights.Delayed == 'true').limit(10) \
    .toPandas()

In [ ]:
flights.count()

# Préparation du modèle

## Séparation entrainement / test

In [ ]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline

#trainingData = flights.filter(flights.Year.isin('2007'))
#testData = flights.filter(flights.Year == '2008')
(trainingData, testData) = [dataset.cache() for dataset in flights.filter(flights.Year.isin('2007', '2008')).randomSplit([0.7, 0.3])]

In [ ]:
trainingData.limit(10).toPandas()

In [ ]:
trainingData.groupBy('UniqueCarrier').count().orderBy(F.col('count').desc()).toPandas()

In [ ]:
testData.limit(10).toPandas()

In [ ]:
testData.groupBy('UniqueCarrier').count().orderBy(F.col('count').desc()).toPandas()

## Feature engineering

In [ ]:
labelIndexer = StringIndexer(inputCol='Delayed', outputCol='DelayedIndex')
categoricalIndexers = [ StringIndexer(inputCol=inputColName, outputCol=inputColName + 'Index', handleInvalid='skip') \
                       for inputColName in ['Month', 'DayOfWeek', 'UniqueCarrier', 'FlightNum', 'Origin', 'Dest'] ]

In [ ]:
assembler = VectorAssembler(
    inputCols=['MonthIndex', 'DayOfWeekIndex', 'DepTime', 'UniqueCarrierIndex', 'FlightNumIndex', 'OriginIndex', 'DestIndex'],
    outputCol='features')

## Paramétrage du modèle

In [ ]:
rf = RandomForestClassifier(labelCol="DelayedIndex", featuresCol="features", maxBins=8000)

In [ ]:
pipeline = Pipeline(stages=[labelIndexer, *categoricalIndexers, assembler, rf])

model = pipeline.fit(trainingData)

In [ ]:
testPredictions = model.transform(testData)
#testPredictions.filter(testPredictions.prediction == 0).limit(10).toPandas()
testPredictions.limit(10).toPandas()

## Évaluation du modèle

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol='DelayedIndex')
evaluator.evaluate(testPredictions)


In [ ]:
model.stages

In [ ]:
import pandas as pd
rfModel = model.stages[8]
featureImportance = pd.DataFrame({'feature': features, 'importance': rfModel.featureImportances.toArray()})
featureImportance

In [ ]:
import plotly.plotly as py
import cufflinks as cf
featureImportance.iplot(x='feature', kind='barh')

# Exploitation du modèle

In [ ]:
model.save('/data/airplane-model')

# Tuning des hyperparamètres

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [ ]:
rf.getNumTrees()

In [ ]:
paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20, 50, 100]) \
    .addGrid(rf.maxDepth, [3, 5, 7]) \
    .build()

In [ ]:
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(labelCol='DelayedIndex'),
                          numFolds=10)

In [ ]:
cvModel = crossval.fit(trainingData)

In [ ]:
cvModel.bestModel

In [ ]:
rfModel = cvModel.bestModel.stages[6]
rfModel.getNumTrees

In [ ]:
rfModel = cvModel.bestModel.stages[6]
featureImportance = pd.DataFrame({'feature': features, 'importance': rfModel.featureImportances.toArray()})
featureImportance

In [ ]:
featureImportance.iplot(x='feature', kind='barh')

In [ ]:
testPredictions = cvModel.transform(testData)

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol='DelayedIndex')
evaluator.evaluate(testPredictions)

In [ ]:
print(rf.explainParams())